In [100]:
# # reading harvardx data from the csv
# # still a work in progress since their dataset is malformatted
# import csv

# # load data
# csvfile_path = '../datasets/corpus_HarvardX_LatestCourses_based_on_2016-10-18.csv'

# with open(csvfile_path, 'r') as csvfile:
#     reader = csv.DictReader(csvfile)
#     print reader.next()

In [2]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
# dictionary of results
results = udacity_se.search("android")
print results

[{'slug': u'developing-android-apps--ud853', 'title': u'Developing Android Apps'}, {'slug': u'new-android-fundamentals--ud851', 'title': u'New Android Fundamentals'}, {'slug': u'android-for-beginners--ud834', 'title': u'Android for Beginners'}, {'slug': u'android-for-beginners--ud834', 'title': u'Android for Beginners'}, {'slug': u'android-tv-and-google-cast-development--ud875B', 'title': u'Android TV and Google Cast Development'}, {'slug': u'gradle-for-android-and-java--ud867', 'title': u'Gradle for Android and Java'}, {'slug': u'android-wear-development--ud875A', 'title': u'Android Wear Development'}, {'slug': u'material-design-for-android-developers--ud862', 'title': u'Material Design for Android Developers'}, {'slug': u'android-basics-user-input--ud836', 'title': u'Android Basics: User Input'}, {'slug': u'android-basics-user-input--ud836', 'title': u'Android Basics: User Input'}]


In [4]:
import gensim.models as models
import gensim.models.word2vec as word2vec

# prepare corpus and ngram reader
# create
# corpus = word2vec.Text8Corpus('datasets/enwik8')
# ngram_phrases= models.Phrases(corpus)
# ngram = models.phrases.Phraser(ngram_phrases)
# ngram.save('models/phraser/phraser')
# or just load from file
ngram = models.phrases.Phraser.load('models/phraser/phraser')

# load model
# either or
# model = word2vec.Word2Vec(ngram[corpus], workers=8)
# model.save('models/word2vec/word2vec')
model = word2vec.Word2Vec.load('models/word2vec/word2vec')

# test model
# BUG: this doesn't work that well :(
# it worked pretty well before i put in ngrams
print model.most_similar(positive=['autism'], topn=10)

[(u'[[Vishnu]]', 0.4175434112548828), (u'rulers]]', 0.4047089219093323), (u'Hillary', 0.3996689021587372), (u'International_Conference', 0.39594656229019165), (u'[[Children', 0.3944295048713684), (u'understandable', 0.3920005261898041), (u'[[Harley', 0.38948845863342285), (u'Protocol&lt;br&gt;', 0.3832295536994934), (u'Lovecraft:', 0.3831127882003784), (u'align=left', 0.38207781314849854)]


In [2]:
# wikipedia rewriting
import requests
from lxml import etree


class WikipediaRewriter:
    WIKI_BASE = 'https://en.wikipedia.org/w/api.php?format=xml&action=query&prop=categories&titles='
    
    def clean_category(self, x):
        return x.replace('Category:','')
    
    def rewrite(self, term):
        """
        Given a base term, returns a list of related terms based on the Wikipedia
        category API.
        """
        api = self.WIKI_BASE + term
        r = requests.get(api)
        tree = etree.fromstring(r.text)
        # TODO: ignore wikipedia internal categories
        # https://en.wikipedia.org/wiki/Category:Tracking_categories
        return [self.clean_category(x.get('title')) for x in tree.findall('.//cl')]

    
test_terms = [
    'confirmation bias',
    'Albert Einstein',
    'biochemistry',
]

rewriter = WikipediaRewriter()

for term in test_terms:
    print rewriter.rewrite(term)

['CS1 maint: Unfit url', 'Cognitive biases', 'Cognitive inertia', 'Critical thinking', 'Design of experiments', 'Error', 'Featured articles', 'Ignorance', 'Inductive fallacies', 'Logical fallacies']
['1879 births', '1955 deaths', '20th-century American engineers', '20th-century American writers', '20th-century German writers', '20th-century Jews', '20th-century physicists', 'AC with 19 elements', 'Activists from New Jersey', 'Albert Einstein']
['Biochemistry', 'Biotechnology', 'CS1 maint: Uses editors parameter', 'Molecular biology', 'Pages using ISBN magic links', 'Wikipedia articles with GND identifiers', 'Wikipedia articles with LCCN identifiers']


In [16]:
# compare query rewriting to the normal results 

flatten = lambda l: [item for sublist in l for item in sublist]

searches = [
    "neural networks",
    "swift",
    "venture capital",
    "object-oriented programming",
    "deep learning",
    "macOS"
]

for query in searches:
    # normal results
    num_plain_results = len(udacity_se.search(query))
    
    # with wikipedia rewriting
    wiki_rewritten_queries = rewriter.rewrite(query) + [query]
    wiki_results = [udacity_se.search(q) for q in wiki_rewritten_queries]
    # flatten them
    flattened_wiki_results = flatten(wiki_results)
    num_wiki_results = len(flattened_wiki_results)
    
    print (query, num_plain_results, num_wiki_results)

('neural networks', 2, 2)
('swift', 9, 9)
('venture capital', 1, 1)
('object-oriented programming', 6, 6)
('deep learning', 1, 1)
('macOS', 0, 0)
